# Business Logic Deployments via MMS on OCI Data Science

This notebook walks through how to:
- create a model version set, used for version control of models in the model catalog
- create a model entry for a model using business logic (i.e. not using an ML model)
- create a single model deployment for a model using business logic
- create a model group for logic based models
- create a model group artifact for logic based models
- create a model group history for model groups, this helps with version control and cataloging
- create a model deployment for a model group
- update a model group history
- perform a live update of a model group deployment

Whilst this example is simple, it can be used as a foundation for automating deployment and re-deployment of logic based model deployments via multi-model serving on OCI Data Science

A note on directory structure...

When creating the artifacts for both the model catalog, and the model group artifact bare in mind the directory structure for the multi-model serving will need to look like this:

```model-group
    runtime.yaml -- for the group
    score.py -- for the group
    model1.py -- artifact for model 1
    model2.py -- artifact for model 2
        model1
            runtime.yaml
            score.py
            model.pickle
            model1.py
        model2
            runtime.yaml
            score.py
            model.pickle
            model2.py
```
We need to add model1.py and model2.py to the model_group.zip file because we're adding custom method imports to our model. It may be that you do not need to add model1.py and model2.py to the individual models for MMS, but for single-model deployments you do need to so it is worth adding the file to the zip and model catalog even if there is some redundancy here.

In [1]:
import oci
import ads
import tempfile
from ads.model import GenericModel
from ads.model import ModelVersionSet
from ads.common.model_metadata import UseCaseType

  from ads.common.model_metadata import UseCaseType



### 0: Create Model Version Set

In [ ]:
project_ocid = 'ocid1.datascienceproject...'
compartment_ocid = 'ocid1.compartment...'

In [ ]:
# Create a model version set
mvs1 = ModelVersionSet(
    name = "business-model-live-1",
    description = "business model mvs1")
mvs1.with_compartment_id(compartment_ocid).with_project_id(project_ocid).create()

In [ ]:
# Create a model version set
mvs2 = ModelVersionSet(
    name = "business-model-live-2",
    description = "business model mvs2")
mvs2.with_compartment_id(compartment_ocid).with_project_id(project_ocid).create()

### 1: Deploy Custom Logic as Models

#### Model 1

In [5]:
from model1 import model
import pickle

In [15]:
# basic model to square a given number

generic_model1 = GenericModel(artifact_dir='model1_live')

In [16]:
generic_model1.prepare(inference_conda_env='generalml_p311_cpu_x86_64_v1',
                      model_file_name='score.py',
                      score_py_uri='custom_score_v2.py',
                      inference_python_version='3.11',force_overwrite=True)

  warnings.warn("slug will be deprecated. Provide conda pack path instead.")

ERROR:root:Error occurred in attempt to extract the list of the service conda environments from the object storage for bucket 'service-conda-packs' and namespace 'id19sfcrra6z'. Please make sure that you've provided correct bucket and namespace.
INFO:ADS:To auto-extract taxonomy metadata the model must be provided. Supported models: keras, lightgbm, pytorch, sklearn, tensorflow, pyspark, and xgboost.     ?, ?it/s]


algorithm: null
artifact_dir:
  /Users/hsnart/Documents/Custom Demos and Blogs/Business Logic MMS/model1_live:
  - - .model-ignore
    - runtime.yaml
    - model.pickle
    - test_json_output.json
    - model1.py
    - score.py
framework: null
model_deployment_id: null
model_id: null

In [17]:
m = model()

with open('model1_live/model.pickle','wb') as f:
    pickle.dump(m,f)

In [18]:
generic_model1.verify({'number':5})

Start loading model.pickle from model directory /Users/hsnart/Documents/Custom Demos and Blogs/Business Logic MMS/model1_live ...
Model is successfully loaded.
trying to predict using pickle
data received by predict function
input number is 5
yhat is currently 25


{'prediction': 25}

In [19]:
generic_model1.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
          Available     Serialized model                                                  
          Done          Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [20]:
%%writefile model1_live/runtime.yaml

MODEL_ARTIFACT_VERSION: '3.0'
MODEL_DEPLOYMENT:
  INFERENCE_CONDA_ENV:
    INFERENCE_ENV_PATH: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/General_Machine_Learning_for_CPUs_on_Python_3.11/1.0/generalml_p311_cpu_x86_64_v1
    INFERENCE_ENV_SLUG: generalml_p311_cpu_x86_64_v1
    INFERENCE_ENV_TYPE: data_science
    INFERENCE_PYTHON_VERSION: '3.11'
MODEL_PROVENANCE:
  PROJECT_OCID: ''
  TENANCY_OCID: ''
  TRAINING_CODE:
    ARTIFACT_DIRECTORY: ''
  TRAINING_COMPARTMENT_OCID: ''
  TRAINING_CONDA_ENV:
    TRAINING_ENV_PATH: ''
    TRAINING_ENV_SLUG: ''
    TRAINING_ENV_TYPE: ''
    TRAINING_PYTHON_VERSION: ''
  TRAINING_REGION: ''
  TRAINING_RESOURCE_OCID: ''
  USER_OCID: ''
  VM_IMAGE_INTERNAL_ID: ''

Overwriting model1_live/runtime.yaml


In [21]:
generic_model1.reload_runtime_info()

In [22]:
# Save the model and add it to the model version set
model_id = generic_model1.save(compartment_id=compartment_ocid,
    project_id=project_ocid,
    display_name="Business Model 1",
    model_version_set=mvs1,
    version_label="Version 2")

Start loading model.pickle from model directory /Users/hsnart/Documents/Custom Demos and Blogs/Business Logic MMS/model1_live ...
Model is successfully loaded.
['.model-ignore', 'runtime.yaml', 'model.pickle', 'test_json_output.json', 'model1.py', 'score.py']


loop1:   0%|          | 0/4 [00:00<?, ?it/s]

In [23]:
generic_model1.predict({'number':5},local=True)

Start loading model.pickle from model directory /Users/hsnart/Documents/Custom Demos and Blogs/Business Logic MMS/model1_live ...
Model is successfully loaded.
trying to predict using pickle
data received by predict function
input number is 5
yhat is currently 25


{'prediction': 25}

In [ ]:
single_mdl = generic_model1.deploy(display_name='Business Model 1',deployment_log_group_id='ocid1.loggroup...',deployment_access_log_id='ocid1.log...',deployment_predict_log_id='ocid1.log...')

In [25]:
generic_model1.predict({'number':5},local=False)

{'prediction': 25}

In [26]:
# call model directly using OCI
import requests
from oci.signer import Signer

config = oci.config.from_file("~/.oci/config") 
auth = Signer(
  tenancy=config['tenancy'],
  user=config['user'],
  fingerprint=config['fingerprint'],
  private_key_file_location=config['key_file'],
  pass_phrase=config['pass_phrase'])

endpoint = single_mdl.url+'/predict'
body = {'number':5}
headers = {} 
requests.post(endpoint, json=body, auth=auth, headers=headers).json()

{'prediction': 25}

#### Model 2

In [27]:
from model2 import model
import pickle

In [28]:
# basic model to square a given number

generic_model2 = GenericModel(artifact_dir='model2_live')

In [29]:
generic_model2.prepare(inference_conda_env='generalml_p311_cpu_x86_64_v1',
                      model_file_name='score.py',
                     score_py_uri='custom_score_v2.py',
                      inference_python_version='3.11',force_overwrite=True)

  warnings.warn("slug will be deprecated. Provide conda pack path instead.")

ERROR:root:Error occurred in attempt to extract the list of the service conda environments from the object storage for bucket 'service-conda-packs' and namespace 'id19sfcrra6z'. Please make sure that you've provided correct bucket and namespace.
INFO:ADS:To auto-extract taxonomy metadata the model must be provided. Supported models: keras, lightgbm, pytorch, sklearn, tensorflow, pyspark, and xgboost.     ?, ?it/s]


algorithm: null
artifact_dir:
  /Users/hsnart/Documents/Custom Demos and Blogs/Business Logic MMS/model2_live:
  - - .model-ignore
    - runtime.yaml
    - score.py
framework: null
model_deployment_id: null
model_id: null

In [30]:
m = model()

with open('model2_live/model.pickle','wb') as f:
    pickle.dump(m,f)

In [31]:
generic_model2.verify({'number':9})

Start loading model.pickle from model directory /Users/hsnart/Documents/Custom Demos and Blogs/Business Logic MMS/model2_live ...
Model is successfully loaded.
trying to predict using pickle
data received by predict function
input number is 9
yhat is currently 3.0


{'prediction': 3.0}

In [32]:
generic_model2.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
          Available     Serialized model                                                  
          Done          Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [35]:
%%writefile model2_live/runtime.yaml

MODEL_ARTIFACT_VERSION: '3.0'
MODEL_DEPLOYMENT:
  INFERENCE_CONDA_ENV:
    INFERENCE_ENV_PATH: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/General_Machine_Learning_for_CPUs_on_Python_3.11/1.0/generalml_p311_cpu_x86_64_v1
    INFERENCE_ENV_SLUG: generalml_p311_cpu_x86_64_v1
    INFERENCE_ENV_TYPE: data_science
    INFERENCE_PYTHON_VERSION: '3.11'
MODEL_PROVENANCE:
  PROJECT_OCID: ''
  TENANCY_OCID: ''
  TRAINING_CODE:
    ARTIFACT_DIRECTORY: ''
  TRAINING_COMPARTMENT_OCID: ''
  TRAINING_CONDA_ENV:
    TRAINING_ENV_PATH: ''
    TRAINING_ENV_SLUG: ''
    TRAINING_ENV_TYPE: ''
    TRAINING_PYTHON_VERSION: ''
  TRAINING_REGION: ''
  TRAINING_RESOURCE_OCID: ''
  USER_OCID: ''
  VM_IMAGE_INTERNAL_ID: ''

Overwriting model2_live/runtime.yaml


In [36]:
generic_model2.reload_runtime_info()

In [37]:
# Save the model and add it to the model version set
model_id = generic_model2.save(compartment_id=compartment_ocid,
    project_id=project_ocid,
    display_name="Business Model 2",
    model_version_set=mvs2,
    version_label="Version 1")

Start loading model.pickle from model directory /Users/hsnart/Documents/Custom Demos and Blogs/Business Logic MMS/model2_live ...
Model is successfully loaded.
['.model-ignore', 'runtime.yaml', 'model.pickle', 'model2.py', 'score.py']


loop1:   0%|          | 0/4 [00:00<?, ?it/s]

In [38]:
generic_model2.predict({'number':9},local=True)

Start loading model.pickle from model directory /Users/hsnart/Documents/Custom Demos and Blogs/Business Logic MMS/model2_live ...
Model is successfully loaded.
trying to predict using pickle
data received by predict function
input number is 9
yhat is currently 3.0


{'prediction': 3.0}

### 3: Create Model Group Artifact

In [39]:
%%writefile model_group_live/runtime.yaml

MODEL_ARTIFACT_VERSION: '3.0'
MODEL_DEPLOYMENT:
  INFERENCE_CONDA_ENV:
    INFERENCE_ENV_PATH: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/General_Machine_Learning_for_CPUs_on_Python_3.11/1.0/generalml_p311_cpu_x86_64_v1
    INFERENCE_ENV_SLUG: generalml_p311_cpu_x86_64_v1
    INFERENCE_ENV_TYPE: data_science
    INFERENCE_PYTHON_VERSION: '3.11'
MODEL_PROVENANCE:
  PROJECT_OCID: ''
  TENANCY_OCID: ''
  TRAINING_CODE:
    ARTIFACT_DIRECTORY: ''
  TRAINING_COMPARTMENT_OCID: ''
  TRAINING_CONDA_ENV:
    TRAINING_ENV_PATH: ''
    TRAINING_ENV_SLUG: ''
    TRAINING_ENV_TYPE: ''
    TRAINING_PYTHON_VERSION: ''
  TRAINING_REGION: ''
  TRAINING_RESOURCE_OCID: ''
  USER_OCID: ''
  VM_IMAGE_INTERNAL_ID: ''

Writing model_group_live/runtime.yaml


In [40]:
%%writefile model_group_live/score.py
import os
import sys
import json
import pandas as pd
import numpy as np
from functools import lru_cache
from io import BytesIO
import base64
import logging
import cloudpickle

model_name = 'model.pickle'


"""
   Inference script. This script is used for prediction by scoring server when schema is known.
"""


@lru_cache(maxsize=10)
def load_model(model_folder):
    """
    Loads model from the serialized format

    Returns
    -------
    model:  a model instance on which predict API can be invoked
    """
    model_file_name = "model.pickle"
    model_path = model_folder + '/' + model_file_name
    print(f"Model path is: {model_path}")
    if not os.path.exists(model_path):
        error = "Model Path doesn't exist : " + model_path
        print(f"Model path does not exists: {model_path}")
        raise Exception(error)

    # TODO: Load the model from the model_dir using the appropriate loader
    # Below is a sample code to load a model file using `cloudpickle` which was serialized using `cloudpickle`
    # from cloudpickle import cloudpickle
    with open(model_path, "rb") as file:
        model = cloudpickle.load(file)

    print("Model is successfully loaded")
    return model


@lru_cache(maxsize=1)
def fetch_data_type_from_schema(input_schema_path=os.path.join(os.path.dirname(os.path.realpath(__file__)), "input_schema.json")):
    """
    Returns data type information fetch from input_schema.json.

    Parameters
    ----------
    input_schema_path: path of input schema.

    Returns
    -------
    data_type: data type fetch from input_schema.json.

    """
    data_type = {}
    if os.path.exists(input_schema_path):
        schema = json.load(open(input_schema_path))
        for col in schema['schema']:
            data_type[col['name']] = col['dtype']
    else:
        print("input_schema has to be passed in in order to recover the same data type. pass `X_sample` in `ads.model.framework.sklearn_model.SklearnModel.prepare` function to generate the input_schema. Otherwise, the data type might be changed after serialization/deserialization.")
    return data_type

def deserialize(data, input_schema_path):
    """
    Deserialize json serialization data to data in original type when sent to predict.

    Parameters
    ----------
    data: serialized input data.
    input_schema_path: path of input schema.

    Returns
    -------
    data: deserialized input data.

    """

    if isinstance(data, bytes):
        logging.warning(
            "bytes are passed directly to the model. If the model expects a specific data format, you need to write the conversion logic in `deserialize()` yourself."
        )
        return data

    data_type = data.get('data_type', '') if isinstance(data, dict) else ''
    json_data = data.get('data', data) if isinstance(data, dict) else data

    if "numpy.ndarray" in data_type:
        load_bytes = BytesIO(base64.b64decode(json_data.encode('utf-8')))
        return np.load(load_bytes, allow_pickle=True)
    if "pandas.core.series.Series" in data_type:
        return pd.Series(json_data)
    if "pandas.core.frame.DataFrame" in data_type or isinstance(json_data, str):
        return pd.read_json(json_data, dtype=fetch_data_type_from_schema(input_schema_path))
    if isinstance(json_data, dict):
        return pd.DataFrame.from_dict(json_data)

    return json_data



def pre_inference(data, input_schema_path):
    """
    Preprocess data

    Parameters
    ----------
    data: Data format as expected by the predict API of the core estimator.
    input_schema_path: path of input schema.

    Returns
    -------
    data: Data format after any processing.

    """
    data = deserialize(data, input_schema_path)
    return data

def post_inference(yhat):
    """
    Post-process the model results

    Parameters
    ----------
    yhat: Data format after calling model.predict.

    Returns
    -------
    yhat: Data format after any processing.

    """
    return yhat.tolist()

def predict(data, model, input_schema_path=os.path.join(os.path.dirname(os.path.realpath(__file__)), "input_schema.json")):
    """
    Returns prediction given the model and data to predict

    Parameters
    ----------
    model: Model instance returned by load_model API
    data: Data format as expected by the predict API of the core estimator. For eg. in case of sckit models it could be numpy array/List of list/Pandas DataFrame
    input_schema_path: path of input schema.

    Returns
    -------
    predictions: Output from scoring server
        Format: {'prediction': output from model.predict method}

    """
    #input = pre_inference(data, input_schema_path)
    #yhat = post_inference(model.predict(input))
    yhat = model.predict(data)
    return {'prediction': yhat}

Writing model_group_live/score.py


In [53]:
!zip -r model_group_live.zip model_group_live/

updating: model_group_live/ (stored 0%)
updating: model_group_live/runtime.yaml (deflated 53%)
updating: model_group_live/score.py (deflated 67%)
  adding: model_group_live/model2.py (deflated 56%)
  adding: model_group_live/model1.py (deflated 56%)


### 4: Create Model Group

In [54]:
config = oci.config.from_file()

# Initialize client
data_science_client =oci.data_science.DataScienceClient(config)

In [55]:
create_model_group_response = data_science_client.create_model_group(
    create_base_model_group_details=oci.data_science.models.CreateModelGroupDetails(
        create_type="CREATE",
        compartment_id=compartment_ocid,
        project_id=project_ocid,
        model_group_details=oci.data_science.models.HomogeneousModelGroupDetails(
            type="HOMOGENEOUS"),
        member_model_entries=oci.data_science.models.MemberModelEntries(
            member_model_details=[
                oci.data_science.models.MemberModelDetails(
                    model_id=generic_model1.model_id,
                    inference_key="square"),
                    oci.data_science.models.MemberModelDetails(
                    model_id=generic_model2.model_id,
                    inference_key="square-root")]),
        display_name="Business-Model-Group-Live",
        description="Example of creating a Homogenous Model Group on OCI Data Science for custom logic"))

In [56]:
print(create_model_group_response.data.lifecycle_state)

CREATING


In [57]:
model_group_id = create_model_group_response.data.id

### 5: Upload Model Group Artifact

In [58]:
import os

file_name = "model_group_live.zip"
content_disposition = "attachment;filename={}".format(os.path.basename(file_name))

with open(file_name, "rb") as f:
    response = data_science_client.create_model_group_artifact(
        model_group_id=model_group_id,
        model_group_artifact=f,
        content_disposition=content_disposition
    )

In [59]:
print(response.status)

204


### 6: Create Model Group Version Set

In [ ]:
display_name = "Business-Model-Group-Live-v2"
description = "An example model group version set for business logic."

#Create Model Group Version Set details
create_details = oci.data_science.models.CreateModelGroupVersionHistoryDetails(
   compartment_id=compartment_ocid,
    display_name=display_name,
    description=description,
    project_id = project_ocid,
    latest_model_group_id = model_group_id
)

# Create the Model GroupVersion Set
response = data_science_client.create_model_group_version_history(create_details)
print("Created Model GroupVersion Set: ", response.data)

### 7: Deploy Model Group

In [ ]:
# Set Compute Shape 
instance_shape_config_details = oci.data_science.models.ModelDeploymentInstanceShapeConfigDetails(
    memory_in_gbs=16,
    ocpus=1
)

# Set Instance Type
instance_configuration = oci.data_science.models.InstanceConfiguration(
    instance_shape_name="VM.Standard.E4.Flex",
    model_deployment_instance_shape_config_details=instance_shape_config_details
)

# Set Scaling Policy
scaling_policy = oci.data_science.models.FixedSizeScalingPolicy(
    policy_type="FIXED_SIZE",
    instance_count=1  # Adjust as needed
)

# Set Instance Config Details
infrastructure_config_details = oci.data_science.models.InstancePoolInfrastructureConfigurationDetails(
    infrastructure_type="INSTANCE_POOL",
    instance_configuration=instance_configuration,
    scaling_policy=scaling_policy
)

# Set Environment Details (NOTE: Change this to use BYOC option)
environment_config_details = oci.data_science.models.DefaultModelDeploymentEnvironmentConfigurationDetails(
    environment_configuration_type="DEFAULT",
    environment_variables={"WEB_CONCURRENCY":"1"}
)

# Set Model Group Details
model_group_config_details = oci.data_science.models.ModelGroupConfigurationDetails(
    model_group_id=model_group_id
)

# Set Model Group Deployment Details (infrastructure + model group)
model_group_deployment_config_details = oci.data_science.models.ModelGroupDeploymentConfigurationDetails(
    deployment_type="MODEL_GROUP",
    model_group_configuration_details=model_group_config_details,
    infrastructure_configuration_details=infrastructure_config_details,
    environment_configuration_details=environment_config_details
)

# Set logging details
category_log_details = oci.data_science.models.CategoryLogDetails(
    access=oci.data_science.models.LogDetails(
        log_group_id='ocid1.loggroup...',
        log_id='ocid1.log...'
    ),
    predict=oci.data_science.models.LogDetails(
        log_group_id='ocid1.loggroup...',
        log_id='ocid1.log...'
    )
)

# Create Model Deployment using above
create_model_deployment_details = oci.data_science.models.CreateModelDeploymentDetails(
    display_name='Business Logic Model Group',
    description='Model Group Deployment for business logic',
    compartment_id=compartment_ocid,
    project_id=project_ocid,
    model_deployment_configuration_details=model_group_deployment_config_details,
    category_log_details=category_log_details  # or omit entirely if logging not required
)

response = data_science_client.create_model_deployment(
    create_model_deployment_details=create_model_deployment_details
)


In [62]:
model_group_url = response.data.model_deployment_url+'/predict'

In [63]:
model_deployment_ocid=response.data.id

### 8: Score Model Group Deployment via REST

In [65]:
body = {'number':9} 

headers = {'Content-Type':'application/json',
          'model-key':'square'} # all we change is the model key
requests.post(model_group_url, json=body, auth=auth, headers=headers).json()

{'prediction': 81}

In [66]:
headers = {'Content-Type':'application/json',
          'model-key':'square-root'} # all we change is the model key
requests.post(model_group_url, json=body, auth=auth, headers=headers).json()

{'prediction': 3.0}

### 8: Change Business Model 2 - Update Version Set

Here we make it return the square root of the absolute value of the number

In [67]:
from model3 import model
import pickle

In [68]:
# basic model to square a given number

generic_model3 = GenericModel(artifact_dir='model3_live')

In [69]:
generic_model3.prepare(inference_conda_env='generalml_p311_cpu_x86_64_v1',
                      model_file_name='score.py',
                      score_py_uri='custom_score_v2.py',
                      inference_python_version='3.11',force_overwrite=True)

  warnings.warn("slug will be deprecated. Provide conda pack path instead.")

ERROR:root:Error occurred in attempt to extract the list of the service conda environments from the object storage for bucket 'service-conda-packs' and namespace 'id19sfcrra6z'. Please make sure that you've provided correct bucket and namespace.
INFO:ADS:To auto-extract taxonomy metadata the model must be provided. Supported models: keras, lightgbm, pytorch, sklearn, tensorflow, pyspark, and xgboost.     ?, ?it/s]


algorithm: null
artifact_dir:
  /Users/hsnart/Documents/Custom Demos and Blogs/Business Logic MMS/model3_live:
  - - .model-ignore
    - runtime.yaml
    - score.py
framework: null
model_deployment_id: null
model_id: null

In [70]:
m = model()

with open('model3_live/model.pickle','wb') as f:
    pickle.dump(m,f)

In [71]:
generic_model3.verify({'number':9})

Start loading model.pickle from model directory /Users/hsnart/Documents/Custom Demos and Blogs/Business Logic MMS/model3_live ...
Model is successfully loaded.
trying to predict using pickle
data received by predict function
input number is 9
yhat is currently 3.0


{'prediction': 3.0}

In [72]:
generic_model3.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
          Available     Serialized model                                                  
          Done          Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [73]:
%%writefile model3_live/runtime.yaml

MODEL_ARTIFACT_VERSION: '3.0'
MODEL_DEPLOYMENT:
  INFERENCE_CONDA_ENV:
    INFERENCE_ENV_PATH: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/General_Machine_Learning_for_CPUs_on_Python_3.11/1.0/generalml_p311_cpu_x86_64_v1
    INFERENCE_ENV_SLUG: generalml_p311_cpu_x86_64_v1
    INFERENCE_ENV_TYPE: data_science
    INFERENCE_PYTHON_VERSION: '3.11'
MODEL_PROVENANCE:
  PROJECT_OCID: ''
  TENANCY_OCID: ''
  TRAINING_CODE:
    ARTIFACT_DIRECTORY: ''
  TRAINING_COMPARTMENT_OCID: ''
  TRAINING_CONDA_ENV:
    TRAINING_ENV_PATH: ''
    TRAINING_ENV_SLUG: ''
    TRAINING_ENV_TYPE: ''
    TRAINING_PYTHON_VERSION: ''
  TRAINING_REGION: ''
  TRAINING_RESOURCE_OCID: ''
  USER_OCID: ''
  VM_IMAGE_INTERNAL_ID: ''

Overwriting model3_live/runtime.yaml


In [74]:
generic_model3.reload_runtime_info()

In [75]:
model_id = generic_model3.save(compartment_id=compartment_ocid,
    project_id=project_ocid,
    display_name="Business Model 2",
    model_version_set=mvs2,
    version_label="Version 2")

Start loading model.pickle from model directory /Users/hsnart/Documents/Custom Demos and Blogs/Business Logic MMS/model3_live ...
Model is successfully loaded.
['.model-ignore', 'runtime.yaml', 'model3.py', 'model.pickle', 'score.py']


loop1:   0%|          | 0/4 [00:00<?, ?it/s]

### 9: Create New Model Group 

In [76]:
!zip -r model_group_live.zip model_group_live/
# add model3.py to the zip

updating: model_group_live/ (stored 0%)
updating: model_group_live/runtime.yaml (deflated 53%)
updating: model_group_live/score.py (deflated 67%)
updating: model_group_live/model2.py (deflated 56%)
updating: model_group_live/model1.py (deflated 56%)
  adding: model_group_live/model3.py (deflated 55%)


In [77]:
config = oci.config.from_file()

# Initialize client
data_science_client =oci.data_science.DataScienceClient(config)

In [78]:
create_model_group_response = data_science_client.create_model_group(
    create_base_model_group_details=oci.data_science.models.CreateModelGroupDetails(
        create_type="CREATE",
        compartment_id=compartment_ocid,
        project_id=project_ocid,
        model_group_details=oci.data_science.models.HomogeneousModelGroupDetails(
            type="HOMOGENEOUS"),
        member_model_entries=oci.data_science.models.MemberModelEntries(
            member_model_details=[
                oci.data_science.models.MemberModelDetails(
                    model_id=generic_model1.model_id,
                    inference_key="square"),
                    oci.data_science.models.MemberModelDetails(
                    model_id=generic_model3.model_id, # update model to model3 from model2
                    inference_key="square-root")]),
        display_name="Business-Model-Group-Live-v2",
        description="Example of creating a Homogenous Model Group on OCI Data Science for custom logic"))

In [79]:
new_model_group_id = create_model_group_response.data.id

In [80]:
import os

file_name = "model_group_live.zip"
content_disposition = "attachment;filename={}".format(os.path.basename(file_name))

with open(file_name, "rb") as f:
    response = data_science_client.create_model_group_artifact(
        model_group_id=new_model_group_id,
        model_group_artifact=f,
        content_disposition=content_disposition
    )

### 10: Update Model Group Version Set

In [ ]:
display_name = "Business-Model-Group-Live"
description = "An example model group version set for business logic."

#Create Model Group Version Set details
update_details = oci.data_science.models.UpdateModelGroupVersionHistoryDetails(
    display_name=display_name,
    description=description,
    latest_model_group_id = new_model_group_id)

# Create the Model GroupVersion Set
response = data_science_client.update_model_group_version_history(model_group_version_history_id='ocid1.dscmodelgroupversionhistory...',update_model_group_version_history_details=update_details)
print("Created Model GroupVersion Set: ", response.data)

### 10: Live Update of Model Group

In [90]:
update_model_group_configuration_details = oci.data_science.models.UpdateModelGroupConfigurationDetails( model_group_id=new_model_group_id )

In [91]:
model_deployment_configuration_details = oci.data_science.models.UpdateModelGroupDeploymentConfigurationDetails(
 deployment_type="MODEL_GROUP",
 update_type="LIVE",
 model_group_configuration_details=update_model_group_configuration_details
 )

In [92]:
update_model_deployment_details = oci.data_science.models.UpdateModelDeploymentDetails(
 display_name="Business-Logic-Updated",
 description="Live model update to deployment",
 model_deployment_configuration_details=model_deployment_configuration_details
 )

In [93]:
response = data_science_client.update_model_deployment(
 model_deployment_id=model_deployment_ocid, # from the model deployment response object above
 update_model_deployment_details=update_model_deployment_details
 ) 
print("Update submitted. Status:", response.status)

Update submitted. Status: 202


## Supplementary information on automation task

You can use the below to help productionise the creation of new Model Groups and subsequent Live Updates of the server

### 1: How to get Latest OCID of Model in Model Version Set

In [94]:
mvs = ModelVersionSet.from_name(name='business-model-live-1',compartment_id=compartment_ocid)

In [95]:
num_models = len(mvs.models())

In [96]:
print('there are ',num_models,'in model version set')

there are  2 in model version set


In [ ]:
for model in mvs.models():
    print(model)

### 2: How to Get latest OCI of Model Group History

In [ ]:
# get model group id of latest model group in version history

def find_latest_model_group_id(compartment_ocid, project_ocid, group_history_name):
    config = oci.config.from_file()
    data_science_client = oci.data_science.DataScienceClient(config)
    try:
        response =data_science_client.list_model_group_version_histories(
            compartment_id=compartment_ocid,
           project_id=project_ocid,
            sort_by="timeCreated",
            sort_order="DESC"
        )
        # Filter inPython
        filtered = [
            h for h in response.data
            if h.display_name == group_history_name
       ]
        if not filtered:
            print("No model group version histories found with given display name.")
            return None
        history = filtered[0]
        target_model_history = history.id
        response =data_science_client.get_model_group_version_history(model_group_version_history_id=target_model_history)
        model_history = response.data

        return {'history_id': history.id, 'history_name':history.display_name,'latest_model_group_id':model_history.latest_model_group_id}
    except oci.exceptions.ServiceError as e:
        print(f"ServiceError: {e}")
        return None


find_latest_id = find_latest_model_group_id(compartment_ocid,project_ocid,'Business-Model-Group-Live')
print(find_latest_id)
